### Data Extraction of COVID-19 spread

In [2]:
from IPython.core.display import display, HTML,display_html
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np

In [4]:
confirmed_df= pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [5]:
confirmed_df.head()

Province/State Country/Region      Lat     Long  1/22/20  1/23/20  1/24/20  \
0            NaN    Afghanistan  33.0000  65.0000        0        0        0   
1            NaN        Albania  41.1533  20.1683        0        0        0   
2            NaN        Algeria  28.0339   1.6596        0        0        0   
3            NaN        Andorra  42.5063   1.5218        0        0        0   
4            NaN         Angola -11.2027  17.8739        0        0        0   

   1/25/20  1/26/20  1/27/20  ...  5/27/20  5/28/20  5/29/20  5/30/20  \
0        0        0        0  ...    12456    13036    13659    14525   
1        0        0        0  ...     1050     1076     1099     1122   
2        0        0        0  ...     8857     8997     9134     9267   
3        0        0        0  ...      763      763      764      764   
4        0        0        0  ...       71       74       81       84   

   5/31/20  6/1/20  6/2/20  6/3/20  6/4/20  6/5/20  
0    15205   15750   16509   17267   18054   18969  
1     1137    1143    1164    1184    1197    1212  
2     9394    9513    9626    9733    9831    9935  
3      764     765     844     851     852     852  
4       86      86      86      86      86      86  

[5 rows x 140 columns]

In [6]:
dates = confirmed_df.columns[4:]

confirmed_df_long = confirmed_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Confirmed'
)
deaths_df_long = deaths_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
)
recovered_df_long = recovered_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Recovered'
)

In [7]:
confirmed_df_long.sample(5)

Province/State Country/Region      Lat      Long     Date  Confirmed
32804            NaN  Cote d'Ivoire   7.5400   -5.5471  5/24/20       2376
31330            NaN       Tanzania  -6.3690   34.8888  5/18/20        509
31812            NaN         Mexico  23.6345 -102.5528  5/20/20      56594
12156            NaN        Romania  45.9432   24.9668   3/7/20          9
17342         Fujian          China  26.0789  117.9874  3/27/20        331

In [8]:
## Remove Canada from df as recovered data is counted by Country-wise rather than Province/State-wise
recovered_df_long = recovered_df_long[recovered_df_long['Country/Region']!='Canada']

In [9]:
# Merging confirmed_df_long and deaths_df_long
full_table = confirmed_df_long.merge(
  right=deaths_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)
# Merging full_table and recovered_df_long
full_table = full_table.merge(
  right=recovered_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

In [10]:
full_table['Date'] = pd.to_datetime(full_table['Date'])

In [11]:
## Check for missing values
full_table.isna().sum()

Province/State    25160
Country/Region        0
Lat                   0
Long                  0
Date                  0
Confirmed             0
Deaths                0
Recovered          3808
dtype: int64

In [12]:
full_table['Recovered'] = full_table['Recovered'].fillna(0)

In [13]:
## Identify data of ship rows
ship_rows = full_table['Province/State'].str.contains('Grand Princess') | full_table['Province/State'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('MS Zaandam')
full_ship = full_table[ship_rows]

In [14]:
## Remove rows containing COVID 19 spread on ships
full_table = full_table[~(ship_rows)]

In [16]:
# Active Case = confirmed - deaths - recovered
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']
full_table.sample(5)

Province/State Country/Region      Lat      Long       Date  \
22697              NaN        Croatia  45.1000   15.2000 2020-04-16   
33378              NaN       Honduras  15.2000  -86.2419 2020-05-26   
12506              NaN         Angola -11.2027   17.8739 2020-03-09   
27755              NaN        Czechia  49.8175   15.4730 2020-05-05   
7992   South Australia      Australia -34.9285  138.6007 2020-02-21   

       Confirmed  Deaths  Recovered  Active  
22697       1791      35      529.0  1227.0  
33378       4401     188      493.0  3720.0  
12506          0       0        0.0     0.0  
27755       7896     257        0.0  7639.0  
7992           2       0        2.0     0.0

In [17]:
# aggregate data into Country/Region wise and group them by Date and Country/Region.
full_grouped = full_table.groupby(['Date', 'Country/Region'])[['Confirmed', 'Deaths', 'Recovered', 'Active']].sum().reset_index()
full_grouped.sample(5)

Date Country/Region  Confirmed  Deaths  Recovered  Active
1641  2020-01-30        Somalia          0       0        0.0     0.0
18573 2020-04-30       Suriname         10       1        8.0     1.0
10484 2020-03-18        Grenada          0       0        0.0     0.0
9818  2020-03-14        Senegal         10       0        1.0     9.0
3311  2020-02-08   Sierra Leone          0       0        0.0     0.0

In [18]:
# new cases 
temp = full_grouped.groupby(['Country/Region', 'Date' ])[['Confirmed', 'Deaths', 'Recovered']]
temp = temp.sum().diff().reset_index()
mask = temp['Country/Region'] != temp['Country/Region'].shift(1)
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan
# renaming columns
temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']
# merging new values
full_grouped = pd.merge(full_grouped, temp, on=['Country/Region', 'Date'])
# filling na with 0
full_grouped = full_grouped.fillna(0)
# fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
full_grouped[cols] = full_grouped[cols].astype('int')
# 
full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)

In [19]:
## Save cleaned data to a local .csv file
full_grouped.to_csv('COVID-19-time-series-clean-complete.csv', index = False)
full_grouped.sample(5)

Date    Country/Region  Confirmed  Deaths  Recovered   Active  \
10084 2020-03-16  Congo (Kinshasa)          2       0        0.0      2.0   
5690  2020-02-21         Mauritius          0       0        0.0      0.0   
16802 2020-04-21             Gabon        156       1       16.0    139.0   
24588 2020-06-02             China      84161    4638    76370.0   3153.0   
22910 2020-05-24            Canada      86092    6533        0.0  79559.0   

       New cases  New deaths  New recovered  
10084          0           0              0  
5690           0           0              0  
16802         36           0              9  
24588          7           0              1  
22910        955          68              0

### Make Interactive Line Plot with DateSlider

In [20]:
df_country = full_grouped[full_grouped['Country/Region'] == 'US']
df_country.sample(5)

Date Country/Region  Confirmed  Deaths  Recovered    Active  \
7798  2020-03-03             US        118       7        7.0     104.0   
13564 2020-04-03             US     275798    9260     9707.0  256831.0   
15052 2020-04-11             US     526776   24366    31270.0  471140.0   
14494 2020-04-08             US     428654   18270    23559.0  386825.0   
14122 2020-04-06             US     366317   13894    19581.0  332842.0   

       New cases  New deaths  New recovered  
7798          20           1              0  
13564      31790        1322            706  
15052      29930        2009           2480  
14494      31533        2079           1796  
14122      29515        1519           2133

In [21]:
import time
from datetime import datetime

import pandas as pd
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider, DateSlider,  HoverTool, Range1d
from bokeh.plotting import figure
from bokeh.themes import Theme
from bokeh.io import show, output_notebook

output_notebook()

Loading BokehJS ...

In [27]:
## Group by date and aggregate
global_count_df = full_grouped.groupby(['Date'])[['Confirmed', 'Deaths', 'Recovered']]
global_count_df = global_count_df.sum().reset_index()
global_count_df.sample(5)

Date  Confirmed  Deaths  Recovered
15  2020-02-06      30794     634     1391.0
64  2020-03-26     528997   24777   119116.0
100 2020-05-01    3344823  238603  1020966.0
35  2020-02-26      80683    2766    28673.0
29  2020-02-20      75563    2245    17124.0

In [28]:
# global_count_df['Date'] =  global_count_df['Date'].dt.strftime('%d-%m-%Y')
global_count_df['Date'] = pd.to_datetime(global_count_df['Date'],dayfirst = True)
# global_count_df['Date'] = global_count_df['Date'].astype('datetime64[D]')
first_date = global_count_df['Date'][0]
last_date = global_count_df['Date'][len(global_count_df)-1]
global_count_df.set_index('Date', drop = True, inplace = True)
print('Global COVID Spread FROM:',first_date.strftime('%d-%m-%Y'),'TO:',last_date.strftime('%d-%m-%Y'))
global_count_df.sample(5)

Global COVID Spread FROM: 22-01-2020 TO: 05-06-2020


Confirmed  Deaths  Recovered
Date                                    
2020-05-13    4346283  297181  1498044.0
2020-05-26    5588891  350436  2223093.0
2020-02-21      76185    2249    17668.0
2020-01-30       8234     171      136.0
2020-04-18    2316902  163220   575474.0

In [29]:
# Restrict data by date
global_count_df.loc[:'31-01-2020']

Confirmed  Deaths  Recovered
Date                                    
2020-01-22        555      17       28.0
2020-01-23        654      18       30.0
2020-01-24        941      26       35.0
2020-01-25       1434      42       38.0
2020-01-26       2118      56       51.0
2020-01-27       2927      82       58.0
2020-01-28       5578     131      103.0
2020-01-29       6166     133      120.0
2020-01-30       8234     171      136.0
2020-01-31       9927     213      209.0

In [30]:
def bokeh_app(doc):
    df = global_count_df.copy()
    source = ColumnDataSource(data=df)
#     HoverTool(tooltips=[("Date", "@ENTRYDATE{%F}"), ("Transactions", "@Transactions")], formatters={"@ENTRYDATE":"datetime"}, mode='vline')
    hover = HoverTool(names=['covidcount'],tooltips=[("Date", "$x{%d-%m-%Y}"),("Count", "$y{0,000,000}")],formatters={"$x": "datetime"}, mode='mouse')
    hover.point_policy= "snap_to_data"
    plot = figure(title="@Dilip Rajkumar - Global COVID-19 Historic Spread - Bokeh Visualization",
                 plot_width=1200, plot_height=540, x_axis_type='datetime', y_axis_label='Nr. of Cases')
#     plot.x_range=DataRange1d(bounds=(None, 120))
    plot.x_range=Range1d(first_date, last_date)
    plot.y_range=Range1d(10, global_count_df['Confirmed'].max()*1.10)
    plot.add_tools(hover)
    plot.line('Date', 'Confirmed', source=source, color = 'orange',line_width=2, name = 'covidcount', legend_label="Nr. of ConfirmedCases", )
    plot.line('Date', 'Deaths', source=source, color = 'red',line_width=2,name = 'covidcount', legend_label="Nr. of Deaths")
    plot.line('Date', 'Recovered', source=source, color = 'green',line_width=2,name = 'covidcount', legend_label="Nr. of Recovered Cases")
    plot.legend.location = 'top_left'
    
    def callback(attr, old, new):
        date_value =  slider.value
        last_date = datetime.fromtimestamp(date_value/1000).strftime('%d-%m-%Y')
        df_trunc = df.loc[: last_date]
        source.data = df_trunc  

    slider = DateSlider(start=first_date, end=last_date, value='31-01-2020', step=1, title="Date")
    slider.on_change('value', callback)

    doc.add_root(column(slider, plot))

In [31]:
show(bokeh_app)

ERROR:bokeh.server.views.ws:Refusing websocket connection from Origin 'http://localhost:8889';                       use --allow-websocket-origin=localhost:8889 or set BOKEH_ALLOW_WS_ORIGIN=localhost:8889 to permit this; currently we allow origins {'localhost:8888'}


#### References:
1. [COVID 19 - Data Wrangling and Cleaning](https://towardsdatascience.com/covid-19-data-processing-58aaa3663f6)
2. [Bokeh AAPL Stocks - DateTime formatting](http://docs.bokeh.org/en/1.0.2/docs/user_guide/examples/tools_hover_tooltip_formatting.html)